In [1]:
import utils
import metrics
import CNNLSTMModel
import ConvLSTM2DModel

import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

LUCKY_NUMBER = 2
TARGET_SIZE = (32, 32) # For no compression choose -1
TARGET_SLICES = 304

PHOTOS_PATH = "/run/media/student/DataStorage/images/"
MASK_PATH = "/run/media/student/DataStorage/masks/"

2024-12-09 15:08:13.783700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733753293.795070    2346 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733753293.798293    2346 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-09 15:08:13.810888: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
print("GPUs Available: ",tf.config.list_physical_devices('GPU'))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
from tensorflow.keras.mixed_precision import set_global_policy

# Set the policy to mixed precision
set_global_policy('mixed_float16')


Prepare data

In [2]:
np.random.seed(LUCKY_NUMBER)
batch_size = 2
epochs = 10

scan_names = [file[:8] for file in os.listdir(PHOTOS_PATH) if file.endswith(".nii.gz")]
train, val, test = utils.split_train_val_test(scan_names, 0.7, 0.15, 0.15)
print(f"Training data size: {len(train)}, Validation data size: {len(val)}, Test data size: {len(test)}")

print(train)
train_gen = utils.cbct_data_generator(PHOTOS_PATH, MASK_PATH, train)
val_gen = utils.cbct_data_generator(PHOTOS_PATH, MASK_PATH, val)


model = CNNLSTMModel.create_cnn_lstm_model(image_shape=TARGET_SIZE, num_slices=TARGET_SLICES)
#model = ConvLSTM2DModel.create_cnn_convlstm2d_model(image_shape=TARGET_SIZE, num_slices=TARGET_SLICES)
model.summary()

print(model.output_shape)

Training data size: 98, Validation data size: 21, Test data size: 21
['ZEBY_183', 'ZEBY_218', 'ZEBY_107', 'ZEBY_202', 'ZEBY_209', 'ZEBY_130', 'ZEBY_160', 'ZEBY_122', 'ZEBY_117', 'ZEBY_194', 'ZEBY_128', 'ZEBY_136', 'ZEBY_182', 'ZEBY_178', 'ZEBY_232', 'ZEBY_224', 'ZEBY_200', 'ZEBY_201', 'ZEBY_185', 'ZEBY_124', 'ZEBY_179', 'ZEBY_227', 'ZEBY_119', 'ZEBY_162', 'ZEBY_175', 'ZEBY_102', 'ZEBY_157', 'ZEBY_120', 'ZEBY_206', 'ZEBY_110', 'ZEBY_161', 'ZEBY_135', 'ZEBY_141', 'ZEBY_111', 'ZEBY_199', 'ZEBY_187', 'ZEBY_127', 'ZEBY_133', 'ZEBY_237', 'ZEBY_163', 'ZEBY_158', 'ZEBY_174', 'ZEBY_238', 'ZEBY_118', 'ZEBY_190', 'ZEBY_240', 'ZEBY_198', 'ZEBY_109', 'ZEBY_214', 'ZEBY_223', 'ZEBY_150', 'ZEBY_180', 'ZEBY_230', 'ZEBY_228', 'ZEBY_220', 'ZEBY_151', 'ZEBY_177', 'ZEBY_153', 'ZEBY_181', 'ZEBY_167', 'ZEBY_229', 'ZEBY_184', 'ZEBY_233', 'ZEBY_171', 'ZEBY_147', 'ZEBY_169', 'ZEBY_189', 'ZEBY_170', 'ZEBY_222', 'ZEBY_159', 'ZEBY_134', 'ZEBY_207', 'ZEBY_139', 'ZEBY_204', 'ZEBY_152', 'ZEBY_208', 'ZEBY_143', 'ZEBY_

I0000 00:00:1733753298.606532    2346 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4969 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:05:00.0, compute capability: 8.9


Model: "CNN_ConvLSTM_Deconv_Segmentation"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 304, 32, 32, 1) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 304, 32, 32,    │           320 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 304, 16, 16,    │             0 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 304, 16, 16,    │             0 │
│ (TimeDistributed)               │ 32)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 304, 16, 16,    │        18,496 │
│ (TimeDistributed)               │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 304, 8, 8, 64)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 304, 8, 8, 64)  │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 304, 8, 8, 128) │        73,856 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d (ConvLSTM2D)        │ (None, 304, 8, 8, 128) │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, 304, 8, 8, 64)  │       442,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, 304, 16, 16,    │        73,856 │
│ (TimeDistributed)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ (None, 304, 32, 32,    │        73,792 │
│ (TimeDistributed)               │ 64)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_9              │ (None, 304, 32, 32, 1) │            65 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,863,169 (7.11 MB)

 Trainable params: 1,863,169 (7.11 MB)

 Non-trainable params: 0 (0.00 B)

(None, 304, 32, 32, 1)


Train model

In [ ]:
model.layers[9].input.shape

In [ ]:
model.fit(
    train_gen, 
    batch_size=2,
    steps_per_epoch=len(train)//batch_size,
    #validation_data=val_gen,
    #validation_steps=len(val),
    epochs=epochs)

ZEBY_183_0000.nii.gz: (304, 32, 32, 1), ZEBY_183.nii.gz: (304, 32, 32, 1)
ZEBY_218_0000.nii.gz: (304, 32, 32, 1), ZEBY_218.nii.gz: (304, 32, 32, 1)
Batch ready: scans shape (2, 304, 32, 32, 1), masks shape (2, 304, 32, 32, 1)
ZEBY_107_0000.nii.gz: (304, 32, 32, 1), ZEBY_107.nii.gz: (304, 32, 32, 1)
ZEBY_202_0000.nii.gz: (304, 32, 32, 1), ZEBY_202.nii.gz: (304, 32, 32, 1)
Batch ready: scans shape (2, 304, 32, 32, 1), masks shape (2, 304, 32, 32, 1)
Epoch 1/10


I0000 00:00:1733753475.406340    2413 service.cc:148] XLA service 0x7f1dc00bafe0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733753475.406362    2413 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060, Compute Capability 8.9
2024-12-09 15:11:22.666917: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1733753499.837496    2413 cuda_dnn.cc:529] Loaded cuDNN version 90300


In [ ]:
utils.save_model(model)

Make prediction

In [ ]:
test_scan = utils.load_nifti_cbct_scan(PHOTOS_PATH+train[1]+"_0000.nii.gz")
test_scan = test_scan[..., np.newaxis]
test_scan = np.expand_dims(test_scan, axis=0)

predictions = model.predict(test_scan)

predicted_mask = predictions[0]

binary_mask = (predicted_mask > 0.5).astype(np.float32)

print(predicted_mask[200,:,:,0] == predicted_mask[300,:,:,0])

plt.imshow(predicted_mask[120, :, :, 0], cmap="gray")

Plot results

In [ ]:
slice_index = 100

plt.figure(figsize=(10, 5))
plt.subplot(1, 4, 1)
plt.title("Original Slice")
plt.imshow(test_scan[0, slice_index, :, :, 0], cmap="gray")
plt.axis('off')

plt.subplot(1, 4, 2)
plt.title("Predicted Mask")
plt.imshow(predicted_mask[slice_index, :, :, 0], cmap="gray")
plt.axis('off')

plt.subplot(1, 4, 3)
plt.title("Binary Mask")
plt.imshow(binary_mask[slice_index, :, :, 0], cmap="gray")
plt.axis('off')

original_mask = utils.load_nifti_mask(MASK_PATH+train[1]+".nii.gz")
plt.subplot(1, 4, 4)
plt.title("Original Mask")
plt.imshow(original_mask[slice_index,:,:], cmap="gray")
plt.axis('off')
plt.show()